# Preschool in San Mateo.

## 1º Import the libraries 

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import sys
import pandas as pd
from functools import reduce
import operator
import geopandas as gpd
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import funtions as fc

## 2º Query.

In [2]:
df_query_san_mateo = fc.query("preschool",3,1000)

In [3]:
gdf_query_san_mateo = fc.geo_data(df_query_san_mateo)

In [4]:
fc.point_geometry(gdf_query_san_mateo)

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry
0,Safari Kid - San Mateo,37.565645,-122.319028,"{'type': 'Point', 'coordinates': (-122.3190280..."
1,Petite Sorbonne Preschool,37.572255,-122.329470,"{'type': 'Point', 'coordinates': (-122.3294695..."


## 3º Import the query to Mongo in a new collection.

In [5]:
client = MongoClient() 
db = client.ironhack
collection = db.preschool 

In [6]:
collection.create_index ([("geometry", GEOSPHERE)])
data = gdf_query_san_mateo.to_dict(orient='records')
collection.insert_many(data)

## 4º Select the nearest preschool.

In [7]:
db.preschool

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'preschool')

In [8]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[-122.323895,37.566879]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.preschool.find(filt,proyect).limit(1)
preschools = list(result)

In [9]:
preschools

[{'name': 'Safari Kid - San Mateo',
  'latitud': 37.56564537398902,
  'longitud': -122.31902807950975}]

## 5º Map with all preschool. 

In [10]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [11]:
inicial_lat = 37.566879
inicial_long = -122.323895

In [12]:
maps = folium.Map(location = [inicial_lat,inicial_long], zoom_start=15)

In [13]:
San_Mateo = Marker(location = [inicial_lat,inicial_long],tooltip="New office")
San_Mateo.add_to(maps)

In [16]:
for i,row in df_query_san_mateo.iterrows():
    preschool = {
        "location" : [row["latitud"],row["longitud"]],
        "tooltip" : row["name"]    
    }
    if row["latitud"] == 37.56564537398902 and row["longitud"]== -122.31902807950975 :
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "book",
                    icon_color = "black"
        )
    else:
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "book",
                    icon_color = "black"
        )
 
    Marker(**preschool,icon = icon).add_to(maps)
maps